# FFR120 Simulation of Complex Systems - Crowd Disasters
**Group** - Team Butterflies

**Members**:
- Artur ..
- Maria
- Nina
- Ruiqi
- Rundong .,

In [29]:
%matplotlib notebook

import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
import matplotlib.animation as anim
import matplotlib.patches as patches

from sim_state import SimState
from constants import Environment as env
from constants import PersonState as pstate
from constants import SimConstants as sconst

In [30]:
class AnimationSettings:
    display_size = 8, 8   # Size of display in inches
    dpi = 100    # Pixels per inch
    framerate = 15
    exit_color = "#00FF00"
    wall_color = "000000"
    
    def get_marker_size():
        return (
            AnimationSettings.dpi *
            np.mean(AnimationSettings.display_size) *
            sconst.individual_radius /
            np.mean(sconst.max_pos)
        )
    
    
seed = 69420
rng = np.random.default_rng(seed)
gif_writer = anim.PillowWriter(fps=AnimationSettings.framerate)
save_animation = False
save_history = False

## Simulation


In [31]:
def get_environmental_force(individual, state) -> float:
    """Net forces acting on an individual caused by the environment"""
    
    # Compute the displacement between this individual and all others, |p_i - p_j|.
    displacements = state.positions[individual, :] - state.positions

    # The distance to each individual is the norm of each displacement |p_i - p_j|.
    distances = np.linalg.norm(displacements, axis=1)

    # Find all the collisions that occur, which is defined as the instances
    # where the distances are within 2 radiuses, since this means that each
    # perimeter is just barely touching.
    # Exclude the instances where the distance is 0, since this means that
    # the individual is being compared to itself (or something has gone terribly wrong):
    collisions = (
        (distances < 2*sconst.individual_radius) &
        (distances > 0)
    )

    net_environmental_force = np.sum(sconst.collision_rebound / displacements[collisions], axis=0)
    
    return net_environmental_force

In [32]:
def get_desire_force(individual, state) -> float:
    exit = env.exits[0]
    exit_center = exit[0]+(exit[2]-exit[0])/2
    
    displacement = exit_center - state.positions[individual]
    desired_direction = displacement / np.linalg.norm(displacement)
    
    velocity = state.velocities[individual]    
    optimal_velocity = desired_direction*sconst.v_max
    
    desire_force = sconst.individual_force * (optimal_velocity - velocity)
    
    return desire_force

In [33]:
def get_obstacle_forces(state) -> np.ndarray:
    obstacle_forces = np.zeros(shape=sconst.shape_2d)
    
    for individual in range(sconst.n_individuals): 
        x = state.positions[individual,0]
        y = state.positions[individual,1]
        
        if x < sconst.distance_wall:
            d = x - env.walls[3][0][1]
            obstacle_forces[individual, 0] = math.exp(-d/sconst.r_0)
            
        elif x > env.walls[0][1][0] - sconst.distance_wall :
            d = env.walls[1][0][0] - x
            obstacle_forces[individual,0] = -math.exp(-d/sconst.r_0)
            
        if y < sconst.distance_wall:
            d = y - env.walls[3][0][1]
            obstacle_forces[individual,1] = math.exp(-d/sconst.r_0)
            
        elif y > env.walls[1][0][0] - sconst.distance_wall:
            d = env.walls[1][0][0] - y
            obstacle_forces[individual,1] = -math.exp(-d/sconst.r_0)
            
    return obstacle_forces
             
    

In [34]:
def get_environmental_forces(state):
    forces = np.zeros(shape=sconst.shape_2d)
    active = state.person_states == pstate.living
    
    for individual in range(sconst.n_individuals): 
        if not active[individual]:
            continue
                
        env_force = get_environmental_force(individual, state)
        forces[individual] = env_force
        
            
    return forces         

In [35]:
def get_social_forces(state):
    forces = np.zeros(shape=sconst.shape_2d)
    active = state.person_states == pstate.living
    
    for individual in range(sconst.n_individuals): 
        if not active[individual]:
            continue
            
        desire_force = get_desire_force(individual, state)
        forces[individual] = desire_force
    
    obstacle_forces = get_obstacle_forces(state)
    forces += obstacle_forces
    
    return forces
    
        
        

In [36]:
def get_overlap_individuals_with_rectangle(positions, rectangle):
    # First check if bounding box of individual overlaps with the rectangle
    bbox_botleft = positions - sconst.individual_radius
    bbox_topright = positions + sconst.individual_radius
    
    i_rect_overlap = np.where(
        (bbox_botleft[:, 0] > rectangle[0, 0]) &
        (bbox_topright[:, 0] < rectangle[2, 0]) &
        (bbox_botleft[:, 1] > rectangle[0, 1]) &
        (bbox_topright[:, 1] < rectangle[2, 1])
    )
    
    # TODO:
    # Now see if circle overlaps
    # https://www.baeldung.com/cs/circle-line-segment-collision-detection
    
    return i_rect_overlap


def get_touching_individuals_with_rectangle(positions, rectangle):
    # First check if bounding box of individual touches the rectangle
    bbox_botleft = positions - sconst.individual_radius
    bbox_topright = positions + sconst.individual_radius
    
    i_rect_overlap = np.where(
        (bbox_topright[:, 0] > rectangle[0, 0]) &
        (bbox_botleft[:, 0] < rectangle[2, 0]) &
        (bbox_topright[:, 1] > rectangle[0, 1]) &
        (bbox_botleft[:, 1] < rectangle[2, 1])
    )
    
    # TODO:
    # Now see if circle touches
    # https://www.baeldung.com/cs/circle-line-segment-collision-detection
    
    return i_rect_overlap
    

In [37]:
def run_simulation() -> list[SimState]:   
    max_pos = np.array(sconst.max_pos)

    state = SimState(
        positions = (
            sconst.start_margin * max_pos +
            np.random.rand(*sconst.shape_2d) * max_pos * (1-2*sconst.start_margin)
        ),
        velocities=np.zeros(shape=sconst.shape_2d),
        forces=np.zeros(shape=sconst.shape_2d),
        pressures=np.zeros(shape=sconst.shape_1d),
        person_states=np.zeros(shape=sconst.shape_1d)
    )
    
    history = [state.copy()]
    
    for time_step in range(sconst.n_time_steps):
        if (time_step+1) % 100 == 0:
            print(f"Time step: {time_step}/{sconst.n_time_steps-1}")
            
        active = state.person_states == pstate.living
        
        state.positions[active] = state.positions[active] + state.velocities[active] * sconst.time_inc
        #state.forces = np.zeros(shape=shape_2d)
        #state.pressures = np.zeros(shape=shape_1d)
        
        env_forces = get_environmental_forces(state) 
        social_forces = get_social_forces(state)

        state.forces = env_forces + social_forces
        state.pressures = np.linalg.norm(env_forces, axis=1)
        
        #for individual in range(SimConstants.n_individuals): 
         #   if not active[individual]:
          #      continue
                
           # env_force = get_environmental_force(individual, state)
            #state.forces[individual] += env_force
            #state.pressures[individual] += np.linalg.norm(env_force)
            #state.forces[individual] += get_social_force(individual, state)

        for wall, wall_face in zip(env.walls, env.wall_face):
            i_colliding = get_touching_individuals_with_rectangle(state.positions, wall)
            overlap = (
                wall_face[0]*state.velocities[i_colliding, 0] +
                wall_face[1]*state.velocities[i_colliding, 1]
            )
            state.forces[i_colliding] -= wall_face*overlap.T * sconst.collision_rebound
            
            
        state.velocities[active] = (
            state.velocities[active] +
            state.forces[active] * sconst.force_scalar
        )
        
        i_exiting = get_overlap_individuals_with_rectangle(state.positions, env.exits[0])
        state.person_states[i_exiting] = pstate.exited
        
        i_dead = state.pressures > sconst.lethal_pressure
        state.person_states[i_dead] = pstate.dead
                
        history.append(state.copy())

    return history

In [ ]:
history = run_simulation()

Time step: 99/999
Time step: 199/999


## Animation

In [ ]:
def get_time_step(i_frame: int, frame_rate: int, time_inc: float) -> int:
    """Get the time step corresponding to actual time passed of simulation.

    Example:
        50 fps -> 0.02s per frame
        0.01 time_inc -> 2 time steps per frame
        3 frames -> 6 time_steps

    Args:
        i_frame: The frame currently being rendered:
        frame_rate: Number of frames rendered per second.
        time_inc: The time passed between time steps.
    """
    return int((i_frame / frame_rate) / time_inc)


def get_rectangle_pos_and_size_for_patch(rect) -> tuple[np.ndarray, float, float]:
    """Extract position, width, and height from the rectangle.
    
    Each rectangle is defined by the points (topleft, topright, botright, botleft).
    A patch is defined by the (botleft, width, height).
    """
    
    pos = rect[0, :]
    w = rect[2, 0]-rect[0, 0]
    h = rect[2, 1]-rect[0, 1]
    return pos, w, h 


def render_state(i_frame, fig, history, screen_live, screen_dead, screen_exit):
    i_time = get_time_step(i_frame, AnimationSettings.framerate, sconst.time_inc)
    
    if i_time > len(history):
        return screen_live, screen_dead, screen_exit
    #i_time = i_frame
    
    state = history[i_time]
    
    pos_dead = state.positions[state.person_states == pstate.dead]
    screen_dead.set_data(pos_dead[:, 0], pos_dead[:, 1])
    
    pos_exit = state.positions[state.person_states == pstate.exited]
    screen_exit.set_data(pos_exit[:, 0], pos_exit[:, 1])
    
    pos_live = state.positions[state.person_states == pstate.living]
    screen_live.set_data(pos_live[:, 0], pos_live[:, 1])
        
    return screen_live, screen_dead, screen_exit


def render_simulation(history: list[SimState]) -> None:
    fig = plt.figure(
        figsize=AnimationSettings.display_size,
        dpi=AnimationSettings.dpi
    )
    ax = fig.gca()

    screen_live, = ax.plot([], [], 'bo', ms=AnimationSettings.get_marker_size())
    screen_dead, = ax.plot([], [], 'ro', ms=AnimationSettings.get_marker_size())
    screen_exit, = ax.plot([], [], 'go', ms=AnimationSettings.get_marker_size())
    
    for exit in env.exits:
        exit = patches.Rectangle(
            *get_rectangle_pos_and_size_for_patch(exit),
            color=AnimationSettings.exit_color
        )
        ax.add_patch(exit)
        
    for wall in env.walls:
        wall = patches.Rectangle(
            *get_rectangle_pos_and_size_for_patch(wall),
            color=AnimationSettings.wall_color
        )
        ax.add_patch(wall)
    
    ax.set_xlim(0, sconst.max_pos[0])
    ax.set_ylim(0, sconst.max_pos[1])
        
    animation = anim.FuncAnimation(
        fig=fig, func=render_state, frames=sconst.simulation_time*AnimationSettings.framerate,
        fargs=(fig, history, screen_live, screen_dead, screen_exit),
        blit=True, interval=int(1000/AnimationSettings.framerate)
    )
    return animation
    

In [ ]:
animation = render_simulation(history)

## Save data

In [ ]:
if save_animation:
    animation.save("./gifs/animation.gif", writer=gif_writer)

In [ ]:
def history_to_dataframe(history):
    frames = []

    for i in range(sconst.n_time_steps):
        frame = history[i]
        stacked_frame = np.hstack([
            frame.positions,
            frame.velocities,
            frame.forces,
            frame.pressures.reshape(-1, 1),
            frame.person_states.reshape(-1, 1)
        ])
        frames.append(stacked_frame)

    stacked_history = np.vstack(frames)
    stacked_history.shape
    return pd.DataFrame(stacked_history, columns=[
        "position_x", "position_y", "velocity_x", "velocity_y",
        "force_x", "force_y", "pressure", "person_state"
    ])   

In [ ]:
if save_history:
    df = history_to_dataframe(history)
    df.to_csv('./data/data.csv', index=False)